In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

In [2]:
sys.path.insert(0, os.path.abspath('..'))
from UKMovementSensing import hsmm
from milleniumcohort import create_config

In [3]:
config = create_config('../config.yml')
config.create_data_paths()

## Load model

In [4]:
import pickle
with open(config.model_file, 'r') as f:
    model = pickle.load(file=f)

In [5]:
for o in model.obs_distns:
    print(o.params)

{'mu': array([ 0.17527154]), 'sigma': array([[ 0.00392196]])}
{'mu': array([ 0.07079878]), 'sigma': array([[ 0.00235949]])}
{'mu': array([ 0.20253865]), 'sigma': array([[ 0.02030865]])}
{'mu': array([ 0.10350547]), 'sigma': array([[ 0.00459507]])}
{'mu': array([ 0.03792613]), 'sigma': array([[ 0.00093485]])}
{'mu': array([  8.46216802e-06]), 'sigma': array([[  1.44668713e-07]])}
{'mu': array([ 0.01820733]), 'sigma': array([[ 0.00022307]])}
{'mu': array([ 0.00495341]), 'sigma': array([[  1.33194033e-05]])}
{'mu': array([ 0.88758867]), 'sigma': array([[ 0.77873434]])}
{'mu': array([ 0.04622881]), 'sigma': array([[ 0.0017171]])}


In [6]:
means = pd.DataFrame([o.params['mu'] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
means.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [7]:
sigmas = pd.DataFrame([o.params['sigma'][0] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
sigmas.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [8]:
durations = pd.DataFrame([o.params['lmbda'] for o in model.dur_distns], columns=['duration'])
durations['duration_sec'] = ["{} sec".format(int(round(x*5))) for x in durations['duration']]
durations.transpose()

,0,1,2,3,4,5,6,7,8,9
duration,84.3668,81.1717,13.005,46.2295,37.7828,121.348,50.6245,22.4355,2.58093,3.1596
duration_sec,422 sec,406 sec,65 sec,231 sec,189 sec,607 sec,253 sec,112 sec,13 sec,16 sec


In [9]:
# Transition probablilites
pd.DataFrame(model.trans_distn.trans_matrix).style.background_gradient(cmap='BuGn')

In [10]:
concat = pd.concat([means, sigmas], axis=1, keys=['mean', 'sigma'])
concat = concat.reorder_levels(order=[1,0], axis=1)
concat[('duration', 'lmbda')] = durations['duration_sec']

In [11]:
concat

acceleration               duration
          mean         sigma    lmbda
0     0.175272  3.921961e-03  422 sec
1     0.070799  2.359490e-03  406 sec
2     0.202539  2.030865e-02   65 sec
3     0.103505  4.595073e-03  231 sec
4     0.037926  9.348543e-04  189 sec
5     0.000008  1.446687e-07  607 sec
6     0.018207  2.230715e-04  253 sec
7     0.004953  1.331940e-05  112 sec
8     0.887589  7.787343e-01   13 sec
9     0.046229  1.717103e-03   16 sec

In [12]:
concat.transpose().to_clipboard()